In [7]:
#SVM
import pandas as pd
import numpy as np
import random 
import matplotlib.pyplot as plt

from sklearn import svm

dataset = ["iris",
              "Enfermedad_Cardiaca"]

In [8]:
def LeerDatos(filename : str, separa : str, header = True):
    if (header):
        data = pd.read_csv(filename + ".csv", sep =separa, header = 0)
    else:
        data = pd.read_csv(filename+ ".csv", sep = separa, header = None)
    #data = data.sample(frac = 1) #shuffle data
    data = data.sort_values(data.columns[-1])
    return data.to_numpy()

In [9]:
def Normalizar_Datos(data : np.array):
    #normal = np.empty_like(data)
    for i in range (0,np.size(data[0])):
        media = np.mean(data[:,i])
        desvi =np.std(data[:,i])
        data[:,i] = (data[:,i] - media)/desvi
    return data

In [10]:
def Crear_k_folds(data : np.array , k:int, clases: []):
    folds = []
    tot_clase = []
    prop_clase = [] #Acumulado de indices
    pre_fold = []

    m = np.size(data[:,-1]) #numero de datos
    #n = np.size(data[0])
    for i in clases:
        tot_clase.append(np.count_nonzero( data[:,-1] == i))

    prop_clase.append(tot_clase[0])
    for i in range (1, len(tot_clase)):
        prop_clase.append( prop_clase[i-1] + tot_clase[i])

    pos_ini = 0
    for i in range(0, len(clases)):
        pre_fold.append(np.array_split(data[pos_ini:prop_clase[i]], k))
        pos_ini = prop_clase[i]
    
    for i in range (0,k):
        temp = np.empty( (0,np.size(data[0])) )
        for j in range(0,len(clases)):
            temp = np.vstack( (temp,pre_fold[j][i]))
        folds.append(temp)
            
    return folds

In [56]:
def Calcular_Accuracy(X:np.array, y:np.array, model):
    y_calc = model.predict(X)
    #y_calc = OneShot_Salida(y_calc[len(y_calc)-1])
    #print(y)
    #print(y_calc)
    aciertos = 0
    
    for i in range(0, len(y)):
        if (y[i] == y_calc[i]):
            aciertos += 1
    #print(aciertos)
    return aciertos/len(y)
    
def PromedioAccuracy(test:np.array, model, k, clases):
    accu = np.zeros(k)
    for i in range(0,k):
        X_test = test[i][:,:-1]
        #X_test = X_test.astype('float64')
        #X_test = Normalizar_Datos(X_test)

        y_test = TransformacionClases(test[i][:,-1], clases)

        accu[i] = Calcular_Accuracy(X_test, y_test, model)
    return accu.mean()

In [57]:
def TransformacionClases(y:np.array, clases:[]):
    num_clases = len(clases)
    #vec_clases = np.empty((0), dtype = int)
    vec_clases = []
    for i in y:
        idx = clases.index(i)
        vec_clases.append(idx)
    return vec_clases

In [81]:
def CalculoParametrosSVM(folds:[], k:int, C:float, kernel:str , clases:[]):
    arr_costo = []
    arr_theta = []
    arr_test = []
    for test_i in range(0, k):
        test = folds[test_i] 
        train = np.zeros( (0,np.size(folds[0][0])) )
        for train_i in range (0, k):         
            if (train_i == test_i):
                continue
            else:
                train = np.vstack( (train,folds[train_i]) )
            
        #costo = []
        X_train = train[:,:-1]
        #X_train = X_train.astype('float64')
        #X_train = Normalizar_Datos(X_train)

        y_train = TransformacionClases( train[:,-1], clases)
        
        model = svm.SVC(kernel = kernel, C=C, degree=2)
        model.fit(X_train, y_train)
        arr_test.append(test)
      
    return model, arr_test

In [82]:
def BusquedaParametros(folds:[], k, num_clases, clases:[]):
    kernels = ['linear', 'poly', 'rbf']
    coef = np.arange(0.5,5.1,0.5)
    arr_accu = np.empty( (len(kernels),len(coef) ))

    for ker in range(0,len(kernels)):
        for C in range(0, len(coef)):
            model, test = CalculoParametrosSVM(folds, k, coef[C], kernels[ker], clases)

            arr_accu[ker,C] = PromedioAccuracy(test, model, k, clases)

    return pd.DataFrame(arr_accu, index = kernels, columns = coef)

In [83]:
iris = LeerDatos(dataset[0], separa = ',')
clases = ["Setosa", "Versicolor", "Virginica"]
iris_folds = Crear_k_folds(iris, 3, clases)
#theta, arr_costo, arr_test = CalculoParametros(iris_folds, 3, 500, 0.1, 3, 3, 20, clases)


In [84]:
BusquedaParametros(iris_folds, k=3, num_clases=3, clases=clases)

,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
linear,0.993464,0.979575,0.979575,0.979575,0.979575,0.979575,0.979575,0.979575,0.973039,0.973039
poly,0.973856,0.993464,0.993464,0.986520,0.986520,0.986520,0.986520,0.986520,0.979575,0.979575
rbf,0.953840,0.960376,0.967320,0.967320,0.973856,0.993464,0.993464,0.993464,0.993464,0.993464


In [85]:
cardiaca = LeerDatos(dataset[1], separa = '\t')
clases_cardiaca = [0,1]
cardiaca_folds = Crear_k_folds(cardiaca, k=3, clases=clases_cardiaca)

In [86]:
BusquedaParametros(cardiaca_folds, k=3, num_clases=2, clases=clases_cardiaca)

,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
linear,0.848185,0.841584,0.844884,0.844884,0.841584,0.841584,0.838284,0.838284,0.844884,0.844884
poly,0.673267,0.693069,0.696370,0.696370,0.706271,0.709571,0.709571,0.712871,0.706271,0.706271
rbf,0.633663,0.673267,0.689769,0.693069,0.699670,0.706271,0.706271,0.702970,0.712871,0.709571
